In [ ]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-20 03:33:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.63MB/s    in 0.2s    

2020-08-20 03:33:47 (4.63 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://databootcamp-uci.s3-us-west-1.amazonaws.com/E-query.json"
spark.sparkContext.addFile(url)
df = spark.read.json(SparkFiles.get("E-query.json"))
df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attributes: struct (nullable = true)
 |    |    |    |-- ALAND: long (nullable = true)
 |    |    |    |-- AWATER: long (nullable = true)
 |    |    |    |-- BLKGRPCE: string (nullable = true)
 |    |    |    |-- COUNTYFP: string (nullable = true)
 |    |    |    |-- E11B19001e1: long (nullable = true)
 |    |    |    |-- E11B19001e10: long (nullable = true)
 |    |    |    |-- E11B19001e11: long (nullable = true)
 |    |    |    |-- E11B19001e12: long (nullable = true)
 |    |    |    |-- E11B19001e13: long (nullable = true)
 |    |    |    |-- E11B19001e14: long (nullable = true)
 |    |    |    |-- E11B19001e15: long (nullable = true)
 |    |    |    |-- E11B19001e16: long (nullable = true)
 |    |    |    |-- E11B19001e17: long (nullable = true)
 |    |    |    |-- E11B19001e2: long (nullable = true)
 |    |    |    |-- E11B19001e3: long (nullable = true)
 |    |    |    |-- E

In [ ]:
df.dtypes

[('features',
  'array<struct<attributes:struct<ALAND:bigint,AWATER:bigint,BLKGRPCE:string,COUNTYFP:string,E11B19001e1:bigint,E11B19001e10:bigint,E11B19001e11:bigint,E11B19001e12:bigint,E11B19001e13:bigint,E11B19001e14:bigint,E11B19001e15:bigint,E11B19001e16:bigint,E11B19001e17:bigint,E11B19001e2:bigint,E11B19001e3:bigint,E11B19001e4:bigint,E11B19001e5:bigint,E11B19001e6:bigint,E11B19001e7:bigint,E11B19001e8:bigint,E11B19001e9:bigint,E11B19013e1:bigint,E11B19025e1:bigint,E11B19055e2:bigint,E11B19059e2:bigint,E12B19113e1:bigint,E12B20002e1:bigint,E12B20002e2:bigint,E12B20002e3:bigint,E13B19101e1:bigint,FUNCSTAT:string,GEOID:string,GEOID_Data:string,INTPTLAT:string,INTPTLON:string,MTFCC:string,NAMELSAD:string,OBJECTID:bigint,STATEFP:string,Shape__Area:double,Shape__Length:double,TRACTCE:string>,geometry:struct<rings:array<array<array<double>>>>>>'),
 ('fields',
  'array<struct<alias:string,defaultValue:string,domain:string,length:bigint,name:string,sqlType:string,type:string>>'),
 ('geom

In [ ]:
df1 = df.select("features").collect()

In [ ]:
df1

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df2 = df.select("fields").collect()
df2[0][0]

[Row(alias='FID', defaultValue=None, domain=None, length=None, name='OBJECTID', sqlType='sqlTypeOther', type='esriFieldTypeOID'),
 Row(alias='STATEFP', defaultValue=None, domain=None, length=2, name='STATEFP', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='COUNTYFP', defaultValue=None, domain=None, length=3, name='COUNTYFP', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='TRACTCE', defaultValue=None, domain=None, length=6, name='TRACTCE', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='BLKGRPCE', defaultValue=None, domain=None, length=1, name='BLKGRPCE', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='GEOID', defaultValue=None, domain=None, length=12, name='GEOID', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='NAMELSAD', defaultValue=None, domain=None, length=13, name='NAMELSAD', sqlType='sqlTypeOther', type='esriFieldTypeString'),
 Row(alias='MTFCC', defaultValue=None, domain=None, length=5, name='

In [ ]:
df3 = df.select("geometryProperties").collect()
df3

[Row(geometryProperties=Row(shapeAreaFieldName='Shape__Area', shapeLengthFieldName='Shape__Length', units='esriMeters'))]

In [ ]:
df4 = df.select("geometryType").collect()
df4

[Row(geometryType='esriGeometryPolygon')]

In [ ]:
df5 = df.select("globalIdFieldName").collect()
df5

[Row(globalIdFieldName='')]

In [ ]:
df6 = df.select("objectIdFieldName").collect()
df6

[Row(objectIdFieldName='OBJECTID')]

In [ ]:
df7 = df.select("spatialReference").collect()
df7

[Row(spatialReference=Row(latestWkid=4326, wkid=4326))]

In [ ]:
df8 = df.select("uniqueIdField").collect()
df8

[Row(uniqueIdField=Row(isSystemMaintained=True, name='OBJECTID'))]

In [ ]:
temp_df = df1[0][0]
temp_df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
temp_df[0][0].TRACTCE

'086902'

In [ ]:
clean_data = []

In [ ]:
our_array = ["06059063907", "06059063805", "06059063806", "06059063903", "06059063908", "06059063807", "06059063701", "06059063808", "06059063905", "06059063906", "06059063102", "06059063702", "06059063604", "06059063605", "06059063802", "06059063201", "06059063202", "06059063902", "06059063301", "06059062610", "06059063601", "06059063103", "06059063904", "06059063803", "06059063302" ]

In [ ]:
for row in df1[0][0]:
  a = str(row[0].GEOID)
  if a[:-1] in our_array:
    clean_data.append(row[0])

In [ ]:
clean_data

[Row(ALAND=337300, AWATER=0, BLKGRPCE='3', COUNTYFP='059', E11B19001e1=688, E11B19001e10=24, E11B19001e11=34, E11B19001e12=122, E11B19001e13=83, E11B19001e14=47, E11B19001e15=41, E11B19001e16=32, E11B19001e17=40, E11B19001e2=115, E11B19001e3=12, E11B19001e4=27, E11B19001e5=21, E11B19001e6=12, E11B19001e7=11, E11B19001e8=16, E11B19001e9=51, E11B19013e1=62019, E11B19025e1=55804400, E11B19055e2=83, E11B19059e2=40, E12B19113e1=64800, E12B20002e1=38958, E12B20002e2=37778, E12B20002e3=40027, E13B19101e1=424, FUNCSTAT='S', GEOID='060590632013', GEOID_Data='15000US060590632013', INTPTLAT='+33.6480393', INTPTLON='-117.9063376', MTFCC='G5030', NAMELSAD='Block Group 3', OBJECTID=405, STATEFP='06', Shape__Area=488002.08203125, Shape__Length=2966.66302086031, TRACTCE='063201'),
 Row(ALAND=686935, AWATER=0, BLKGRPCE='1', COUNTYFP='059', E11B19001e1=1280, E11B19001e10=85, E11B19001e11=96, E11B19001e12=127, E11B19001e13=262, E11B19001e14=83, E11B19001e15=89, E11B19001e16=247, E11B19001e17=96, E11B1900

In [ ]:
new_df = spark.createDataFrame(clean_data)

In [ ]:
new_df.show()

+------+------+--------+--------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+------------+-------------------+-----------+------------+-----+-------------+--------+-------+---------------+----------------+-------+
| ALAND|AWATER|BLKGRPCE|COUNTYFP|E11B19001e1|E11B19001e10|E11B19001e11|E11B19001e12|E11B19001e13|E11B19001e14|E11B19001e15|E11B19001e16|E11B19001e17|E11B19001e2|E11B19001e3|E11B19001e4|E11B19001e5|E11B19001e6|E11B19001e7|E11B19001e8|E11B19001e9|E11B19013e1|E11B19025e1|E11B19055e2|E11B19059e2|E12B19113e1|E12B20002e1|E12B20002e2|E12B20002e3|E13B19101e1|FUNCSTAT|       GEOID|         GEOID_Data|   INTPTLAT|    INTPTLON|MTFCC|     NAMELSAD|OBJECTID|STATEFP|    Shape__Area|   Shape__Length|TRACTCE|
+-

In [ ]:
panda_df = new_df.toPandas()

In [ ]:
panda_df

,ALAND,AWATER,BLKGRPCE,COUNTYFP,E11B19001e1,E11B19001e10,E11B19001e11,E11B19001e12,E11B19001e13,E11B19001e14,E11B19001e15,E11B19001e16,E11B19001e17,E11B19001e2,E11B19001e3,E11B19001e4,E11B19001e5,E11B19001e6,E11B19001e7,E11B19001e8,E11B19001e9,E11B19013e1,E11B19025e1,E11B19055e2,E11B19059e2,E12B19113e1,E12B20002e1,E12B20002e2,E12B20002e3,E13B19101e1,FUNCSTAT,GEOID,GEOID_Data,INTPTLAT,INTPTLON,MTFCC,NAMELSAD,OBJECTID,STATEFP,Shape__Area,Shape__Length,TRACTCE
0,337300,0,3,059,688,24,34,122,83,47,41,32,40,115,12,27,21,12,11,16,51,62019.0,55804400,83,40,64800.0,38958.0,37778.0,40027.0,424,S,060590632013,15000US060590632013,+33.6480393,-117.9063376,G5030,Block Group 3,405,06,4.880021e+05,2966.663021,063201
1,686935,0,1,059,1280,85,96,127,262,83,89,247,96,61,0,0,40,23,13,34,24,82353.0,130566800,198,101,111375.0,59471.0,54193.0,65625.0,753,S,060590638071,15000US060590638071,+33.6707969,-117.9274147,G5030,Block Group 1,428,06,9.943708e+05,4988.515566,063807
2,209303,0,2,059,321,0,30,29,17,0,0,21,18,6,105,11,54,0,0,30,0,23924.0,17866200,149,28,38708.0,27750.0,32019.0,20625.0,105,S,060590637022,15000US060590637022,+33.6409341,-117.9214514,G5030,Block Group 2,515,06,3.027684e+05,2518.881817,063702
3,323900,0,2,059,417,26,0,31,40,75,77,35,59,0,0,0,47,9,18,0,0,112132.0,62904600,79,46,114635.0,55281.0,54653.0,55408.0,223,S,060590633022,15000US060590633022,+33.6321488,-117.9106383,G5030,Block Group 2,641,06,4.684434e+05,2903.018625,063302
4,323906,0,1,059,370,9,37,54,38,26,7,35,122,7,17,0,0,0,0,9,9,106250.0,51478600,114,56,153846.0,66620.0,NaN,49583.0,232,S,060590633021,15000US060590633021,+33.6349404,-117.9078651,G5030,Block Group 1,754,06,4.684838e+05,2903.205471,063302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,625083,0,3,059,931,44,10,143,161,131,91,42,14,80,0,0,29,60,77,0,49,70795.0,71901100,121,119,60814.0,32212.0,31538.0,38636.0,586,S,060590639063,15000US060590639063,+33.6601360,-117.9041334,G5030,Block Group 3,1734,06,9.046137e+05,4487.184195,063906
72,835355,0,3,059,474,0,29,30,54,41,0,71,0,68,26,0,12,36,44,33,30,44000.0,29781700,103,0,34792.0,23793.0,NaN,21513.0,213,S,060590639023,15000US060590639023,+33.6713146,-117.9129663,G5030,Block Group 3,1804,06,1.209237e+06,4531.540365,063902
73,426535,0,4,059,632,65,11,78,133,37,96,89,13,13,0,37,11,0,0,49,0,79153.0,58369900,89,83,79389.0,34906.0,48671.0,22076.0,370,S,060590639024,15000US060590639024,+33.6781853,-117.9129197,G5030,Block Group 4,1805,06,6.175345e+05,3772.653635,063902
74,548656,0,3,059,613,15,17,61,55,51,42,63,109,18,8,18,25,19,77,20,15,90417.0,64527400,299,205,95750.0,53594.0,55950.0,49018.0,371,S,060590639053,15000US060590639053,+33.6600313,-117.9114501,G5030,Block Group 3,1807,06,7.940139e+05,3692.445287,063905


In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [ ]:
panda_df.to_csv('cleaned_E-query.csv')

In [28]:
!cp cleaned_E-query.csv "drive/My Drive/Final Project"